# Code

In [1]:
import json
import re
import os
import pandas as pd
import asyncio

from subprocess import run
from utils.download import capture_firestore_responses
from utils.parser import parse

YEAR = "2025-2026-Q1"

def display_top(df: pd.DataFrame, filename=None) -> pd.DataFrame:
    out = df.copy()
    out = out.reset_index(drop=True)
    out.index.name = "Rank"
    out.index = out.index + 1
    if filename is not None:
        out.to_csv(f"csv/{YEAR}/{filename}_ranker.csv")
    return out


def create_dir_if_absent(dir: str) -> None:
    if not os.path.exists(dir):
        os.makedirs(dir)


await capture_firestore_responses(output_file=f"db/raw/{YEAR}.txt")
df = parse(f"db/raw/{YEAR}.txt", ['name', 'price_out', 'format', 'degree', 'type', 'available'])

df = df[df["Available"] == True]
df["Ratio"] = (df['Degree'] * df['Volume']) / df['Price']
df = df.drop(columns=["Available"])

df = df.sort_values(by=["Ratio", "Price", "Volume"], ascending=[False, True, False])
df = df.reset_index(drop=True)
df.index.name = "Rank"
df.index = df.index + 1

create_dir_if_absent(f"csv/{YEAR}")
df.to_csv(f"csv/{YEAR}/ranker.csv")
run(["jupyter", "nbconvert", "--to=pdf", "notebook.ipynb", "--output", "top.pdf"], check=True)
# avg time for processing this cell : ~2min

Navigating to https://quinzaine.org...
Intercepting response from: https://firestore.googleapis.com/google.firestore.v1.Firestore/Listen/channel?VER=8&database=projects%2Fquinzaine-3fb2e%2Fdatabases%2F(default)&RID=50211&CVER=22&X-HTTP-Session-Id=gsessionid&zx=pe4c8j4yo0aj&t=1 (Status: 200)
Captured response body chunk (length: 54).
Intercepting response from: https://firestore.googleapis.com/google.firestore.v1.Firestore/Listen/channel?gsessionid=E74L3YzCcW_6na2t2M7ReGlofBRG-HTVCrYubF1lRrw&VER=8&database=projects%2Fquinzaine-3fb2e%2Fdatabases%2F(default)&RID=rpc&SID=jXVuujiXrQ8u4mOw0ZIojQ&AID=0&CI=0&TYPE=xmlhttp&zx=qqifgaete8o7&t=1 (Status: 200)
Intercepting response from: https://firestore.googleapis.com/google.firestore.v1.Firestore/Listen/channel?VER=8&database=projects%2Fquinzaine-3fb2e%2Fdatabases%2F(default)&gsessionid=E74L3YzCcW_6na2t2M7ReGlofBRG-HTVCrYubF1lRrw&SID=jXVuujiXrQ8u4mOw0ZIojQ&RID=50212&AID=5&zx=bobd2gj9eyh3&t=1 (Status: 200)
Captured response body chunk (length: 10)

[NbConvertApp] Converting notebook notebook.ipynb to pdf
[NbConvertApp] Writing 51814 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 53661 bytes to top.pdf


CompletedProcess(args=['jupyter', 'nbconvert', '--to=pdf', 'notebook.ipynb', '--output', 'top.pdf'], returncode=0)

# Top 20, any type of beer

In [2]:
display_top(df[df["Volume"] < 75]).head(20)

,Name,Price,Volume,Degree,Type,Ratio
Rank,,,,,,
1,Kerel Kaishaku Fût,2.5,25,15,Blonde,150.0
2,Kerel Kaishaku,3.5,33,15,Blonde,141.428571
3,Kasteel triple,2.6,33,11,Blonde,139.615385
4,Bush 10,2.5,33,10.5,Blonde,138.6
5,Piraat,2.5,33,10.5,Ambrée,138.6
6,Bush 12 Fût,2.2,25,12,Ambrée,136.363636
7,Chimay rouge,2.2,33,9,Trappiste,135.0
8,Corne du bois des pendus quadruple,3,33,12,Ambrée,132.0
9,Queue de charrue triple,2.3,33,9,Blonde,129.130435


# Top 20, blonde beers

In [3]:
display_top(df[(df["Type"] == "Blonde") & (df["Volume"] < 75)], filename="blonde").head(20)


,Name,Price,Volume,Degree,Type,Ratio
Rank,,,,,,
1,Kerel Kaishaku Fût,2.5,25,15,Blonde,150.0
2,Kerel Kaishaku,3.5,33,15,Blonde,141.428571
3,Kasteel triple,2.6,33,11,Blonde,139.615385
4,Bush 10,2.5,33,10.5,Blonde,138.6
5,Queue de charrue triple,2.3,33,9,Blonde,129.130435
6,Corne du bois des pendus 10 triple,2.8,33,10,Blonde,117.857143
7,Duvel,2.4,33,8.5,Blonde,116.875
8,Triple plaisir (la),2.3,33,8,Blonde,114.782609
9,Carolus triple d'or,2.6,33,9,Blonde,114.230769


# Top 20, ambrées

In [4]:
display_top(df[(df["Type"] == "Ambrée") & (df["Volume"] < 75)], filename="amber").head(20)


,Name,Price,Volume,Degree,Type,Ratio
Rank,,,,,,
1,Piraat,2.5,33,10.5,Ambrée,138.6
2,Bush 12 Fût,2.2,25,12,Ambrée,136.363636
3,Corne du bois des pendus quadruple,3,33,12,Ambrée,132.0
4,Gulden Draak 9000,2.9,33,10.5,Ambrée,119.482759
5,Maredsous 10,3.1,33,10,Ambrée,106.451613
6,Bon secours heritage,2.5,33,8,Ambrée,105.6
7,Troubadour magma,2.9,33,9,Ambrée,102.413793
8,Carolus ambrio,2.6,33,8,Ambrée,101.538462
9,Quintine ambrée,2.9,33,8.5,Ambrée,96.724138


# Top trappistes

In [5]:
display_top(df[(df["Type"] == "Trappiste") & (df["Volume"] < 75)], filename="trapist")

,Name,Price,Volume,Degree,Type,Ratio
Rank,,,,,,
1,Chimay rouge,2.2,33,9,Trappiste,135.0
2,Westmalle triple,2.5,33,9.5,Trappiste,125.4
3,Rochefort triple extra,2.4,33,9,Trappiste,123.75
4,Rochefort 10,3.1,33,11.3,Trappiste,120.290323
5,Rochefort 8,2.7,33,9.2,Trappiste,112.444444
6,Chimay blanche,2.4,33,8,Trappiste,110.0
7,Chimay bleue,3,33,9,Trappiste,99.0
8,Rochefort 6,2.6,33,7.5,Trappiste,95.192308
9,Chimay Verte (150),3.5,33,10,Trappiste,94.285714


# Top 20, brunes

In [6]:
brown_beers = display_top(df[(df["Type"] == "Brune") & (df["Volume"] < 75)], filename="brown").head(20)

# Top 20, fruitées

In [7]:
display_top(df[(df["Type"] == "Fruitée") & (df["Volume"] < 75)], filename="fruit").head(20)


,Name,Price,Volume,Degree,Type,Ratio
Rank,,,,,,
1,Queue de charrue rouge,2.7,33,8.7,Fruitée,106.333333
2,Gauloise fruits rouges,2.6,33,8.2,Fruitée,104.076923
3,Frambush,2.8,33,8.5,Fruitée,100.178571
4,Chouffe cherry Fût,2,25,8,Fruitée,100.0
5,Pêche Mel Bush Fût,2,25,8,Fruitée,100.0
6,Delirium red Fût,2,25,8,Fruitée,100.0
7,Val dieu fruitee,3.2,33,9,Fruitée,92.8125
8,Barbar Rouge,2.9,33,8,Fruitée,91.034483
9,Kasteel red Fût,2.2,25,8,Fruitée,90.909091


# Top blanches

In [8]:
display_top(df[(df["Type"] == "Blanche") & (df["Volume"] < 75)], filename="white")

,Name,Price,Volume,Degree,Type,Ratio
Rank,,,,,,
1,Bon secours prestige,2.8,33,9,Blanche,106.071429
2,St Hubertus Blanche Fût,2,25,7.2,Blanche,90.0
3,Chouffe blanche,2.2,33,6,Blanche,90.0
4,Blanche de Bruxelles,2,33,4.5,Blanche,74.25
5,St bernardus witbier,2.5,33,5.5,Blanche,72.6
6,Boriner vice,2.9,33,6,Blanche,68.275862
7,Blanche de Namur,1.8,25,4.5,Blanche,62.5
8,Troublette,3,33,5.6,Blanche,61.6


# Top 20, 75cl

In [9]:
display_top(df[df["Volume"]  == 75], filename="75cl").head(20)

,Name,Price,Volume,Degree,Type,Ratio
Rank,,,,,,
1,Bush 12,7,75,12,Ambrée,128.571429
2,Moinette brune,5,75,8.5,Brune,127.5
3,Lupulus Hibernatus,5.7,75,9,Brune,118.421053
4,Lupulus blonde,5.5,75,8.5,Blonde,115.909091
5,Lupulus brune,5.7,75,8.5,Brune,111.842105
6,Binchoise brune,5.3,75,7.7,Brune,108.962264
7,Chimay Blanche (Cinq Cents),5.6,75,8,Trappiste,107.142857
8,Lupulus Organicus,6,75,8.5,Blonde,106.25
9,Moinette blonde,6,75,8.5,Blonde,106.25


# Top 50 du rat (à plus que 5° quand même (big up à Hunter))

In [10]:
rat = display_top(df[(df["Degree"] >= 5) & (df["Volume"] < 75) & (df["Price"] <= 2.5)])
rat = rat.sort_values(by=["Ratio", "Volume"], ascending=[False, True])
rat.head(50)

,Name,Price,Volume,Degree,Type,Ratio
Rank,,,,,,
1,Kerel Kaishaku Fût,2.5,25,15,Blonde,150.0
2,Bush 10,2.5,33,10.5,Blonde,138.6
3,Piraat,2.5,33,10.5,Ambrée,138.6
4,Bush 12 Fût,2.2,25,12,Ambrée,136.363636
5,Chimay rouge,2.2,33,9,Trappiste,135.0
6,Queue de charrue triple,2.3,33,9,Blonde,129.130435
7,Westmalle triple,2.5,33,9.5,Trappiste,125.4
8,Rochefort triple extra,2.4,33,9,Trappiste,123.75
9,Duvel,2.4,33,8.5,Blonde,116.875
